# 模型保存与加载

本教程介绍Keras模型的保存和加载方法，这是模型部署和继续训练的关键技能。

## 学习目标

1. 掌握完整模型的保存与加载
2. 了解仅保存权重的方法
3. 理解不同保存格式的区别
4. 学会保存自定义对象

## 1. 环境配置与模型准备

In [ ]:
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 设置随机种子
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

print(f"TensorFlow版本: {tf.__version__}")

In [ ]:
# 准备数据
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, test_size=0.2, random_state=RANDOM_SEED
)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, test_size=0.25, random_state=RANDOM_SEED
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

print(f"训练集: {X_train.shape}")

In [ ]:
# 构建并训练一个简单模型
def create_model():
    """创建一个简单的回归模型"""
    model = keras.Sequential([
        keras.layers.Dense(30, activation='relu', input_shape=[8]),
        keras.layers.Dense(30, activation='relu'),
        keras.layers.Dense(1)
    ])
    model.compile(
        loss='mse',
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        metrics=['mae']
    )
    return model

model = create_model()
model.summary()

In [ ]:
# 训练模型
history = model.fit(
    X_train, y_train,
    epochs=20,
    validation_data=(X_valid, y_valid),
    verbose=1
)

# 评估原始模型
original_loss, original_mae = model.evaluate(X_test, y_test, verbose=0)
print(f"\n原始模型 - MSE: {original_loss:.4f}, MAE: {original_mae:.4f}")

## 2. 保存完整模型

### 方法一：SavedModel格式（推荐）

SavedModel是TensorFlow的标准格式，包含：
- 模型架构
- 模型权重
- 训练配置（优化器、损失函数等）
- 计算图（支持TensorFlow Serving部署）

In [ ]:
# 保存为SavedModel格式（目录）
model_path = 'saved_models/my_model'
model.save(model_path)

print(f"模型已保存到: {model_path}")
print("\n目录结构:")
for root, dirs, files in os.walk(model_path):
    level = root.replace(model_path, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for file in files:
        print(f"{subindent}{file}")

In [ ]:
# 加载SavedModel
loaded_model = keras.models.load_model(model_path)

# 验证加载的模型
loaded_loss, loaded_mae = loaded_model.evaluate(X_test, y_test, verbose=0)
print(f"加载模型 - MSE: {loaded_loss:.4f}, MAE: {loaded_mae:.4f}")
print(f"与原始模型一致: {np.isclose(original_loss, loaded_loss)}")

### 方法二：HDF5格式（.h5）

HDF5是Keras的传统格式，保存为单个文件。

In [ ]:
# 保存为HDF5格式
h5_path = 'saved_models/my_model.h5'
model.save(h5_path)

print(f"模型已保存到: {h5_path}")
print(f"文件大小: {os.path.getsize(h5_path) / 1024:.2f} KB")

In [ ]:
# 加载HDF5模型
loaded_h5_model = keras.models.load_model(h5_path)

# 验证
h5_loss, h5_mae = loaded_h5_model.evaluate(X_test, y_test, verbose=0)
print(f"H5模型 - MSE: {h5_loss:.4f}, MAE: {h5_mae:.4f}")

### 方法三：Keras格式（.keras）

Keras 3.0引入的新格式，结合了SavedModel和HDF5的优点。

In [ ]:
# 保存为.keras格式（如果TensorFlow版本支持）
try:
    keras_path = 'saved_models/my_model.keras'
    model.save(keras_path)
    print(f"模型已保存到: {keras_path}")
except Exception as e:
    print(f"当前版本可能不支持.keras格式: {e}")

## 3. 仅保存和加载权重

当你只需要权重而不需要完整模型结构时，可以单独保存权重。
这在以下场景很有用：
- 迁移学习
- 模型架构需要修改
- 减小存储空间

In [ ]:
# 保存权重
weights_path = 'saved_models/my_weights'
model.save_weights(weights_path)

print(f"权重已保存到: {weights_path}")

In [ ]:
# 创建相同架构的新模型
new_model = create_model()

# 加载权重前的表现（随机初始化）
before_loss, before_mae = new_model.evaluate(X_test, y_test, verbose=0)
print(f"加载权重前 - MSE: {before_loss:.4f}, MAE: {before_mae:.4f}")

# 加载权重
new_model.load_weights(weights_path)

# 加载权重后的表现
after_loss, after_mae = new_model.evaluate(X_test, y_test, verbose=0)
print(f"加载权重后 - MSE: {after_loss:.4f}, MAE: {after_mae:.4f}")

## 4. 使用Checkpoint保存最佳模型

在训练过程中自动保存最佳模型是最佳实践。

In [ ]:
# 创建新模型用于演示
checkpoint_model = create_model()

# 定义检查点回调
checkpoint_path = 'saved_models/best_model.h5'
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    checkpoint_path,
    monitor='val_loss',       # 监控验证损失
    save_best_only=True,      # 只保存最佳模型
    save_weights_only=False,  # 保存完整模型
    verbose=1
)

# 训练模型
history = checkpoint_model.fit(
    X_train, y_train,
    epochs=30,
    validation_data=(X_valid, y_valid),
    callbacks=[checkpoint_cb],
    verbose=1
)

In [ ]:
# 加载最佳模型
best_model = keras.models.load_model(checkpoint_path)

# 比较最终模型和最佳模型
final_loss, final_mae = checkpoint_model.evaluate(X_test, y_test, verbose=0)
best_loss, best_mae = best_model.evaluate(X_test, y_test, verbose=0)

print("模型对比:")
print(f"最终模型 - MSE: {final_loss:.4f}, MAE: {final_mae:.4f}")
print(f"最佳模型 - MSE: {best_loss:.4f}, MAE: {best_mae:.4f}")

## 5. 保存和加载自定义模型

使用子类化API创建的自定义模型需要特殊处理。

In [ ]:
# 定义自定义模型
class CustomModel(keras.Model):
    def __init__(self, units=30, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.dense1 = keras.layers.Dense(units, activation='relu')
        self.dense2 = keras.layers.Dense(units, activation='relu')
        self.output_layer = keras.layers.Dense(1)
    
    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.output_layer(x)
    
    def get_config(self):
        """返回模型配置，用于序列化"""
        config = super().get_config()
        config.update({'units': self.units})
        return config
    
    @classmethod
    def from_config(cls, config):
        """从配置重建模型"""
        return cls(**config)

# 创建并训练自定义模型
custom_model = CustomModel(units=30, name='custom_model')
custom_model.compile(loss='mse', optimizer='adam', metrics=['mae'])
custom_model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid), verbose=1)

In [ ]:
# 保存自定义模型
custom_path = 'saved_models/custom_model'
custom_model.save(custom_path)

# 加载时需要提供自定义类
loaded_custom = keras.models.load_model(
    custom_path,
    custom_objects={'CustomModel': CustomModel}
)

# 验证
custom_loss, custom_mae = loaded_custom.evaluate(X_test, y_test, verbose=0)
print(f"加载的自定义模型 - MSE: {custom_loss:.4f}, MAE: {custom_mae:.4f}")

## 6. 导出用于部署

将模型导出为可用于生产环境的格式。

In [ ]:
# 使用SavedModel格式导出（用于TensorFlow Serving）
export_path = 'saved_models/export/1'  # 版本号目录
model.save(export_path)

print(f"模型已导出到: {export_path}")
print("\n可以使用以下命令启动TensorFlow Serving:")
print(f"tensorflow_model_server --model_base_path={os.path.abspath('saved_models/export')} --rest_api_port=8501")

## 7. 清理保存的文件

In [ ]:
# 列出所有保存的文件
import shutil

print("已保存的模型文件:")
if os.path.exists('saved_models'):
    for item in os.listdir('saved_models'):
        item_path = os.path.join('saved_models', item)
        if os.path.isdir(item_path):
            print(f"  [目录] {item}")
        else:
            size = os.path.getsize(item_path) / 1024
            print(f"  [文件] {item} ({size:.2f} KB)")

# 取消注释以下代码来清理文件
# shutil.rmtree('saved_models')
# print("\n已清理所有保存的文件")

## 小结

### 保存格式选择

| 格式 | 扩展名 | 优点 | 缺点 |
|------|--------|------|------|
| SavedModel | 目录 | TF Serving部署、完整计算图 | 文件较大 |
| HDF5 | .h5 | 单文件、广泛支持 | 无法保存自定义对象的代码 |
| Keras | .keras | 现代格式、易用 | 需要较新版本 |

### 最佳实践

1. **训练时使用ModelCheckpoint**: 自动保存最佳模型
2. **生产部署用SavedModel**: 支持TensorFlow Serving
3. **实验和分享用HDF5**: 单文件便于传输
4. **自定义模型实现get_config**: 确保可序列化